# Slippage in Futures Markets

The paper titled "Slippage in Futures Markets: Evidence from the Sydney Futures Exchange" by Alex Frino and Teddy Oetomo discusses market impact cost, or slippage, in the futures market. Comparisons to equity markets are made instantly and the authors note that slippage in the futures markets is relatively smaller.

A quick introduction to slippage:

In layman's terms, slippage is the difference between the price you expected to pay (if you are buying a stock, the current share price you see on your app) and the price you end up actually paying.

Back to the jargon, although the research for this paper is done on the futures markets, as the authors felt the need to address the literature gap, we will quickly see how their methodolgy applies just as well to the equity markets. The first aim of the paper is to quantify the effects of slippage in futures markets and compare it to that of equity markets. The second goal is to test just how much the features that play a part in slippage calculation in equity markets, explain slippage in the futures markets. Their comparisons help model slippage in the equity market, with a few tweaks to the dataset.

Research on market impact in the equities markets classifies the total price impact into temporary and permanent costs. Temporary price impact is associated with the implicit transaction costs we are familiar with, and permanent price impact is a way to quantify information content of a trade. Exisiting works in this area suggest that block* purchase trades made by institutional investors carry significantly greater permanent price impact than large sell orders. Chan and Lakonishok (1993, 1995) propose two possible explanations for this phenomenon, one being that large purchases carry more information than sales. Another reason may be that the reluctance of downstairs trading brokers to facilitate these large trades suggest they carry less transaction costs (i.e. temporary impact).

This asymmetry is far less prevalent in futures markets for the main reason of an absence of upstairs trading in the futures markets. A lack of anonymity found in upstairs markets, which attract traders signaling they are liquidity traders, the portion of liquidity traders is smaller in downstairs markets, leading to higher information imbalance and adverse selection costs**. Consequently, the total and permanent effects of transactions are lower in markets without upstairs trading, like futures markets. However, it should be noted that while the presence of information asymmetry in the equity markets may make methods of calculating implicit transaction costs ineffective in the futures markets, the converse is not necessarily true.

**A block trade is a high-volume transaction in a security that is privately negotiated and executed outside of the open market for that security. Major broker-dealers often provide "block trading" services—sometimes known as "upstairs trading desks"—to their institutional clients* <br>
***Adverse selection is when sellers have information that buyers do not have, or vice versa, about some aspect of product quality.*

In [51]:
import pandas as pd
import numpy as np

In [52]:
df = pd.read_csv('xnas-itch-20230703.tbbo.csv')
df.head()

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol
0,1688371200660869841,1688371200660704717,1,2,32,T,B,0,194120000000,1,130,165124,303634,193630000000,194120000000,27,27,1,1,AAPL
1,1688371201201402566,1688371201201237816,1,2,32,T,B,0,194110000000,2,130,164750,304724,193900000000,194110000000,5,400,1,1,AAPL
2,1688371201233688992,1688371201233524761,1,2,32,T,B,0,194110000000,8,130,164231,304850,193900000000,194110000000,5,398,1,1,AAPL
3,1688371201317556361,1688371201317392163,1,2,32,T,B,0,194110000000,2,130,164198,305101,193900000000,194110000000,5,390,1,1,AAPL
4,1688371201478520666,1688371201478356044,1,2,32,T,B,0,194000000000,7,130,164622,306430,193900000000,194000000000,5,200,1,1,AAPL


In [53]:
# preprocessing
df['price'] = df['price']/1e9
df['bid_px_00'] = df['bid_px_00']/1e9
df['ask_px_00'] = df['ask_px_00']/1e9

The paper does not use individual trades as the unit measure to compare market-impact costs following under the assumption that large orders are often broken down into smaller trades; instead the authors group sequences of trades into trade packages which are assumed to belong to one large order. Orders are grouped into a package if they are by the same trader, in the same direction, and less than 1 day has passed since the last trade. 

The lack of features in the dataset only allows for groupings in trade directions. Hence slippage can be seen as the cost incurred when trading in a particular direction over the given time period (i.e. buying or selling).

In [59]:
def slippage_calculation(trades, tick_size):
    opening_price = trades.iloc[0]['price']
    closing_price = trades.iloc[-1]['price']      
    vwap = (trades['price'] * trades['size']).sum() / trades['size'].sum()
    
    slippage = (vwap - opening_price) / tick_size
    liquidity_effect = (closing_price - vwap) / tick_size
    information_effect = (closing_price - opening_price) / tick_size
    
    return slippage, liquidity_effect, information_effect

def process_trade_packages(df):
    df['package_id'] = df['side'].astype('category').cat.codes
    grouped = df.groupby('package_id')
    results = []

    for package_id, group in grouped:
        slippage, liquidity_effect, information_effect = slippage_calculation(group, tick_size)
        
        results.append({
            'package_id': package_id,
            'slippage': slippage,
            'liquidity_effect': liquidity_effect,
            'information_effect': information_effect
        })
    
    return pd.DataFrame(results)

tick_size = 1
results_df = process_trade_packages(df)
results_df


,package_id,slippage,liquidity_effect,information_effect
0,0,-1.306923,-0.263077,-1.57
1,1,-1.466647,-0.203353,-1.67
2,2,-1.178435,-0.401565,-1.58


Negative slippage (magintude of loss) is observed for all B, A, and N orders, analogous to our arrival cost calculations in [RL-PPO-v2 (6).ipynb](https://github.com/aryandhr/blockhouse-work-trial/blob/main/RL-PPO-v2%20(6).ipynb)

### References

Frino, A. (2005, October 19). Slippage in futures markets: Evidence from the Sydney Futures Exchange. Wiley Online Library. https://onlinelibrary.wiley.com/doi/abs/10.1002/fut.20184

Adverse Selection
https://www.investopedia.com/terms/a/adverseselection.asp#:~:text=Adverse%20selection%20is%20when%20sellers,they%20will%20collect%20on%20it

Block Trades
https://www.cmegroup.com/education/courses/market-regulation/block-trades/block-trades-what-is-a-block-trade.html